In [4]:
import pandas as pd
import os
import sys

# --- 0. 경로 설정 ---
# (PROJECT_ROOT는 data 폴더의 상위 폴더를 가리킵니다)
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"Added project root to sys.path: {PROJECT_ROOT}")

# 모든 데이터는 'data/processed'에 있다고 가정
DATA_DIR = os.path.join(PROJECT_ROOT, "processed")
FINANCIALS_FILE = os.path.join(DATA_DIR, "summary_quarter2.csv")

print(f"Financials file path: {FINANCIALS_FILE}")

# --- 1. 탐색: 재무 데이터에 어떤 기업/티커가 있는지 확인 ---
print("\n--- 1. 재무 파일 탐색 ---")
try:
    financials_df = pd.read_csv(FINANCIALS_FILE)
    
    # 'ticker' 컬럼을 문자열로 변환 (비교를 위해)
    financials_df['ticker'] = financials_df['ticker'].astype(str)
    
    unique_tickers = financials_df[['corp_name', 'ticker']].drop_duplicates().sort_values('corp_name')
    
    print("파일에 포함된 고유 기업/티커 목록:")
    print(unique_tickers)
    
except FileNotFoundError:
    print(f"[Error] {FINANCIALS_FILE}을 찾을 수 없습니다.")
    print("파일을 data/processed 폴더에 올바르게 배치했는지 확인해주세요.")
    # (이 경우, 이후 단계를 진행할 수 없음)
except Exception as e:
    print(f"[Error] 파일 로드 실패: {e}")

# --- 2. 정의: 우리의 '마스터 티커' 정의 ---
print("\n--- 2. 우리가 사용할 '마스터 티커' (주가 기준) ---")
MASTER_TICKERS = {
    "042660": "HD한국조선해양",
    "009540": "HD현대중공업",
    "010140": "삼성중공업",
    "443060": "한화오션",
    "010620": "현대미포조선",
    "329180": "HD현대마린솔루션"
}
print(MASTER_TICKERS)


# --- 3. 맵핑: '재무 티커' -> '마스터 티커' 딕셔너리 생성 ---
print("\n--- 3. 맵핑 딕셔너리 정의 ---")
# [!!!] 1번 단계의 출력 결과를 보고, 이 맵핑을 완성해야 합니다. [!!!]
#     (재무 티커) : (마스터 티커)
ticker_mapping = {
    '164830': '042660', # HD한국조선해양
    '111704': '443060', # 한화오션
    '126380': '009540', # HD현대중공업
    '126478': '010140', # 삼성중공업
    '164609': '010620', # 현대미포조선
    '1194689':'329180' # 현대마린솔루션
}
print("적용할 맵핑:", ticker_mapping)


# --- 4. 적용 및 검증 ---
if 'financials_df' in locals(): # 1번 단계가 성공했을 경우에만 실행
    print("\n--- 4. 맵핑 적용 및 검증 ---")
    
    # 'ticker' 컬럼의 값을 'master_ticker' 라는 새 컬럼으로 변환
    financials_df['master_ticker'] = financials_df['ticker'].map(ticker_mapping)
    
    print("\n[맵핑 적용 후 상위 5줄 미리보기]")
    print(financials_df[['as_of_date', 'corp_name', 'ticker', 'master_ticker']].head())
    
    # [검증] 맵핑에 실패한 데이터가 있는지 확인 (가장 중요!)
    # (맵핑 딕셔너리에 없는 티커만 추출)
    failed_tickers = financials_df[financials_df['master_ticker'].isna()]['corp_name'].unique()
    
    if len(failed_tickers) > 0:
        print(f"\n[!!!] 맵핑 실패 [!!!]")
        print(f"{failed_tickers} 의 맵핑 정보가 'ticker_mapping' 딕셔너리에 없습니다.")
        print("3번 단계의 딕셔너리를 완성하고 다시 실행해주세요.")
    else:
        print(f"\n[SUCCESS] 모든 재무 데이터가 마스터 티커와 맵핑되었습니다.")
        
        # 'ticker' 컬럼을 'master_ticker'로 교체
        financials_df = financials_df.drop(columns=['ticker', 'corp_name']) # 기존 ticker, corp_name 제거
        financials_df = financials_df.rename(columns={'master_ticker': 'ticker'}) # master_ticker를 'ticker'로 변경
        print(" -> 'ticker' 컬럼을 마스터 티커로 교체 완료.")
        print(financials_df[['as_of_date', 'ticker', 'roe']].head())

Financials file path: /workspace/ship-ai/data/processed/summary_quarter2.csv

--- 1. 재무 파일 탐색 ---
파일에 포함된 고유 기업/티커 목록:
     corp_name   ticker
0     HD한국조선해양   164830
26   HD현대마린솔루션  1194689
31      HD현대미포   164609
57     HD현대중공업   126380
76       삼성중공업   126478
102       한화오션   111704

--- 2. 우리가 사용할 '마스터 티커' (주가 기준) ---
{'042660': 'HD한국조선해양', '009540': 'HD현대중공업', '010140': '삼성중공업', '443060': '한화오션', '010620': '현대미포조선', '329180': 'HD현대마린솔루션'}

--- 3. 맵핑 딕셔너리 정의 ---
적용할 맵핑: {'164830': '042660', '111704': '443060', '126380': '009540', '126478': '010140', '164609': '010620', '1194689': '329180'}

--- 4. 맵핑 적용 및 검증 ---

[맵핑 적용 후 상위 5줄 미리보기]
   as_of_date corp_name  ticker master_ticker
0  2019-05-15  HD한국조선해양  164830        042660
1  2019-08-14  HD한국조선해양  164830        042660
2  2019-11-14  HD한국조선해양  164830        042660
3  2020-03-30  HD한국조선해양  164830        042660
4  2020-05-15  HD한국조선해양  164830        042660

[SUCCESS] 모든 재무 데이터가 마스터 티커와 맵핑되었습니다.
 -> 'ticker' 컬럼을 마스터 티커로 교체 완료.
   as_o

In [5]:
# --- 5. Ticker 맵핑이 완료된 재무 데이터를 새 파일로 저장 ---
if 'financials_df' in locals() and len(failed_tickers) == 0:
    
    # 저장할 새 파일 이름 정의
    CLEANED_FINANCIALS_FILE = os.path.join(DATA_DIR, "financials_mapped.csv")
    
    # [핵심] 'financials_df' 객체를 CSV 파일로 저장
    financials_df.to_csv(CLEANED_FINANCIALS_FILE, index=False)
    
    print(f"\n[SAVE SUCCESS] Ticker 맵핑이 완료된 데이터를 새 파일로 저장했습니다.")
    print(f" -> {CLEANED_FINANCIALS_FILE}")


[SAVE SUCCESS] Ticker 맵핑이 완료된 데이터를 새 파일로 저장했습니다.
 -> /workspace/ship-ai/data/processed/financials_mapped.csv
